# ALS and SDG
### Main Steps
* Initialize $U$ and $V$
* Determine accuracy metrics
* Set up max distance calculation
* Load in matrix 
* Compute solutions to $U$ and $V$
* Monitor Optimization Progress
* Visualize Results

#### Initializing U and V
Random initialization for now


In [1]:
import numpy as np
import pandas as pd

In [ ]:
K = 5 # five latent factors tentatively 
np.random.seed(42)
# I is the number of users <- these will be determined by cluster size
# M is the number of items 
# Can either fill with 1's or random values
U = np.random.uniform(0, 1, size=K*I).reshape((I, K))
V = np.random.uniform(0, 1, size=K*M).reshape((M, K))
Uold = np.zeros_like(U)
Vold = np.zeros_like(V)

#### Accuracy metrics
Choosing RMSE for simplicity as well as setting up max update

In [ ]:
def rmse(X, Y):
    return np.sqrt(np.nanmean((X-Y)**2))

def max_update(X, Y):
    return np.noram(((X-Y)/Y).ravel(), np.inf)

error = [(0, rmse(R, np.inner(U, V)))]

## SGD function
For the stochastic gradient descent function, we make update formulae for the U and V respectively under SDG.

Within SGD the update functions for vectors in the user and item matrices are as follows: 

For all $m, i \in R$ where $R_{m,i}$ is an observed rating and $\alpha$ is the rate parameter,

$U_{i} = U_{i} + \alpha V_{m}(R_{m,i} - <V_{m}, U_{i}>)$

$V_{m} = V_{m} + \alpha U_{i}(R_{m,i} - <V_{m}, U_{i}>)$

Within the error function we pass in the inner product of $V$ and $U$, $\hat{R}$, along with ratings matrix $R$ to calculate the RMSE. RMSE was calculated as follows:

For vectors $x_i \in R$, $y_i \in \hat{R}$, $\text{RMSE}(R, \hat{R}) =  \left[\frac{1}{n}\sum_{i=1}^{n} \|x_i - y_i\|_2^2 \right]^{1/2}$

In [ ]:
def standard_SGD(U, V, R, error, update, rate=0.1, max_iterations=300, threshold=0.001):
    """
    Performs stochastic gradient descent on user and item matrices U and V optimizing the RMSE. 
    """
    # Optimize over 300 iterations
    for iteration in range(1, max_iterations): # starting from one due to first iteration being hardcoded
        for m, i in zip(*np.where(~np.isnan(R))): # might want to change to not is nan but is zero as that is where no ratings are
            U[i] = U[i] + rate*V[m]*(R.iloc[m,i] - np.inner(V[m], U[i]))
            V[m] = V[m] + rate*U[i]*(R.iloc[m,i] - np.inner(V[m], U[i]))
        error += [(t, rmse(R, np.inner(V,U)))]
    return U, V, error


error = pd.DataFrame(error, columns=['iteration', 'rmse'])

## ALS function
For the alternating least squares function we make some slight adjustments, instead of updating each vector we update separate feature matrices $U$ $V$ and handle regularization terms later on, updates are as follows:

Load in U and V initalized to all 1s
Create 2 hashmaps, 1 containing user index as keys, and indices to articles they've rated as values. The other containing article index as keys, and index to users that have rated them as values.

To create the hashmaps, when we load in our matrix we have to do a larger calculation iterating through all the rows, and then for every row going through the columns finding their values. OR we can just use a sparse matrix which means a lot less iteration during the algorithm and more up front. 

Then next steps would be to make sparse representation of the matrix (there are functions for this in scipy, or we can do it ourselves)

Once we have this we then do the fixing of a matrix and the optimization of the others

for i in 1 to n/m:
* U[i] = (sum: V[hash index] * V[hash index]^transpose)^-1 * R[indices in V that correspond to items that User i has rated] * V[hash index]
* V[j] = (sum: U[hash index] * U[hash index]^transpose)^-1 * R[indices in U that correspond to users that rated item j] * U[hash index]

Highlight then is that we can just use the matrices like above, with indices related to the user, now we could also do some up front hashing with a sparse representation of the matrix, or we could also just see if the sparse matrix works that way?


Ah it makes sense now, yi yi^t is a column vector times a row, or n x 1 times 1 x n, which is therefore an NxN matrix, which when subtracted normalization lambda identity makes so much more sense
The interesting thing then becomes, can I just multiply the sub-matrix of V times its transpose and get the same result?

We need a matrix LOL just using a hash table is a good way to go, lets make some psuedocode below to get a better idea of how we could make a good hashtable 

We'd want two hashtables:

user table, needs to store for every user the items that they interacted with as index and then we access scores with indexing on the matrix and we could consider using touples as well to keep their associated score
item table, needs to store all users for every item that interacted with it as well as their ratings

these tables should be local so that they arent too memory hungry.

For this we would just have to activate a for loop and iterate over stuff with the zip function filling out ratings similarly to the other matrix functions that are already established. And would we want to do this with the data frame while we compose R? That way we can compose R V and U at the same time? We could also just nested forloop that bitch, or use np.flatten on the matrix 

## Matrix function testing


In [ ]:
def create_big_boy():
    full = pd.DataFrame()
    for i in range(4):
        df = pd.read_csv(f"../MIND_large/csv/tensorflow_dataset_chunk{i}.csv", index_col=0)
        full = pd.concat([full, df])
    return full
full_tf = create_big_boy()
news_text = pd.read_csv('../MIND_large/csv/news_cluster_labels.csv')
all_ratings2 = full_tf.groupby('user_id')['news_id'].apply(list).reset_index()
scores = full_tf.groupby('user_id')['score'].apply(list).reset_index()
all_ratings2['scores'] = scores['score']

Now we want to create our matrices while respecting the need for hash maps for the ALS algorithm 
We need a map of 'item' index and user indexes to optimize V
We need a map of 'user' index and item indexes to optimize U 
When we create the item cluster matrix we are taking the dataset as well as the number of users or items depending on our clustering method

So generally speaking for our map building, 
For item index and the user indices, we initialize a map of all 'items' as indices, and since were quantifying users by their index after grouping them together we can just append them to a list at the item index whenever they show up
For user index and item indices, in the for loop all we have to do is check if the user is there and if not we add them but if so we add their item index, which for item clustering is just their cluster, under user clustering with normal items we would have to find each items index and then after having each items index we would need to see when a cluster rates something we'd attach that index to their list, and then add that cluster to the item index list in the other table, so tbh lots of hash maps wow 

In [1]:
import matrix_modules
import pandas as pd
import numpy as np

In [2]:
ratings, news, users = matrix_modules.load_dataset()

Empty DataFrame
Columns: []
Index: [oid sha256:a9c27e2669c351fda4471b981356d4574c81cebb3455ccd683990cb7a3edc305, size 2030881682, oid sha256:1e6d5c7fd8579951d88fb601f7d41f397fbea17de0345aaa84f0a3d68c04ed70, size 1961173597]


KeyError: 'user_id'